In [1]:
import pandas as pd
import os,glob
import re

folder_path = 'data'

list_score = []
list_card = []
list_num_card = []
list_result = []
list_save_time = []
list_run_time = []
list_main_class = []
list_sub_class = []
list_main_index = []
list_sub_index = []
list_ascension = []
list_num_battle = []
list_enable_dlc = []
list_relic = []
list_num_relic = []

def fillInClassStr(string):
    if 'fda62ada-520e-42f3-aa88-e4a78549c4a2' in string:
        return 'Melting Remnant'
    if '9317cf9a-04ec-49da-be29-0e4ed61eb8ba' in string:
        return 'Stygian Guard' 
    if '4fe56363-b1d9-46b7-9a09-bd2df1a5329f' in string:
        return 'Umbra'
    if 'c595c344-d323-4cf1-9ad6-41edc2aebbd0' in string:
        return 'Hellhorned'
    if '46ae87db-d92e-4fcb-a3bc-67c723d7bebd' in string:
        return 'Wurmkin'
    if 'fd119fcf-c2cf-469e-8a5a-e9b0f265560d' in string:
        return 'Awoken'
    return None

for filename in glob.glob(os.path.join(folder_path, '*')):
    with open(filename, 'r') as f:
        text = f.read()
        score_text = re.findall(r'\"score\":[0-9]+', text)
        score = int(re.findall(r'[0-9]+', score_text[0])[0])
        list_score.append(score)

        card_text = re.findall(r'\"cardDataID\":\"[0-9|a-z|-]+\"', text)
        num_of_card = len(card_text)
        card_list = []
        for card_str in card_text :
            result = re.findall(r'[0-9|a-z|-]{8}-[0-9|a-z|-]+', card_str)
            card_list.append(result[0])
        list_card.append(card_list)
        list_num_card.append(num_of_card)

        result_text = re.findall(r'\"victory\":[a-z]+',text)
        win = False
        if 'true' in result_text[0]:
            win = True
        list_result.append(win)

        save_time_text = re.findall(r'\"saveDate\":\"[0-9]+-[0-9]+-[0-9]+ [0-9]+:[0-9]+:[0-9]+',text)
        save = re.findall(r'[0-9]+-[0-9]+-[0-9]+ [0-9]+:[0-9]+:[0-9]+',save_time_text[0])[0]
        save_time = pd.to_datetime(save, format='%Y-%m-%d %H:%M:%S')
        list_save_time.append(save_time)

        run_time_text = re.findall(r'\"runTime\":[0-9]+',text)
        run_time = int(re.findall(r'[0-9]+',run_time_text[0])[0])/60
        list_run_time.append(run_time)

        ascension_text = re.findall(r'\"ascensionLevel\":[0-9]+', text)
        ascension = int(re.findall(r'[0-9]+',ascension_text[0])[0])
        list_ascension.append(ascension)

        classes = re.findall(r'\"className\":\"[0-9|a-z|-]+\"',text)
        main_class = re.findall(r'[0-9|a-z|-]{8}-[0-9|a-z|-]+', classes[0])[0]
        sub_class = re.findall(r'[0-9|a-z|-]{8}-[0-9|a-z|-]+', classes[1])[0]

        main_class = fillInClassStr(main_class)
        sub_class = fillInClassStr(sub_class)

        list_main_class.append(main_class)
        list_sub_class.append(sub_class)

        classes_index = re.findall(r'\"championIndex\":[0|1]',text)
        main_index = 0
        sub_index = 0
        if len(classes_index) != 0 :
            if '1' in classes_index[0]:
                main_index = 1
            if '1' in classes_index[1]:
                sub_index = 1
        list_main_index.append(main_index)
        list_sub_index.append(sub_index)

        battle_text = re.findall(r'\"numBattlesWon\":[0-9]+',text)
        num_battle = re.findall(r'[0-9]+',battle_text[0])
        list_num_battle.append(num_battle[0])

        dlc_text = re.findall(r'\"enabledDlcs\":\[*\]',text)
        dlc = False
        if len(classes_index) != 0:
            dlc = True
        list_enable_dlc.append(dlc)

        relic_list_tmp = re.findall(r'\"relicDataID\":\"[0-9|a-z|-]+\"', text)
        relic_list = []
        for relic_str in relic_list_tmp :
            result = re.findall(r'[0-9|a-z|-]{8}-[0-9|a-z|-]+', relic_str)
            relic_list.append(result[0])
        num_of_relic = len(relic_list)
        list_relic.append(relic_list)
        list_num_relic.append(num_of_relic)
    
card_data = pd.DataFrame({'ascension': list_ascension, 'score': list_score, 'result':list_result, 'DLC': list_enable_dlc,'num_battle':list_num_battle, 'num_of_cards':list_num_card,'time': list_save_time,'duration':list_run_time,
                            'num_of_relic':list_num_relic, 'main_class':list_main_class,'sub_class':list_sub_class, 'main_index':list_main_index, 'sub_index': list_sub_index,'card_list': list_card, 'relic_list':list_relic})

card_data = card_data.sort_values(by=['time'])


In [2]:
card_data = card_data.drop(card_data[card_data.duration < 5].index)
card_data = card_data.drop(card_data[card_data.main_class == None].index)
card_data = card_data.drop(card_data[card_data.sub_class == None].index)
card_data = card_data.drop(card_data[card_data.num_of_cards >50].index)
card_data.to_csv("clean.csv",index=False)

In [3]:
card_list_stat = pd.read_csv("card.csv")
relic_list_stat = pd.read_csv("relic.csv")

card_list_stat['count_win'] = 0
card_list_stat['count_total'] = 0
relic_list_stat['count_win'] = 0
relic_list_stat['count_total'] = 0

for index, row in card_data.iterrows():
    tmp_list = []
    for x in row['card_list']: 
        if x not in tmp_list:
            tmp_list.append(x)
    for card in tmp_list:
        card_index = card_list_stat.index[card_list_stat.CardID == card]
        card_list_stat.loc[card_index, 'count_total'] += 1
        if row['result'] == True:
            card_list_stat.loc[card_index, 'count_win'] += 1
    for relic in row['relic_list']:
        relic_index = relic_list_stat.index[relic_list_stat.ID == relic]
        relic_list_stat.loc[relic_index, 'count_total'] += 1
        if row['result'] == True:
            relic_list_stat.loc[relic_index, 'count_win'] += 1


In [4]:
card_list_stat['winRate'] = card_list_stat['count_win']/card_list_stat['count_total']
relic_list_stat['winRate'] = relic_list_stat['count_win']/relic_list_stat['count_total']

card_list_stat = card_list_stat.drop(card_list_stat[card_list_stat.count_total < 5].index)
card_list_stat.to_csv("card_stat.csv",index=False)
relic_list_stat.to_csv("relic_stat.csv",index=False)

In [5]:

clan_data = pd.DataFrame()
clan_data['main_class'] = card_data['main_class']
clan_data['sub_class'] = card_data['sub_class']
clan_data['main_index'] = card_data['main_index']
clan_data['sub_index'] = card_data['sub_index']
list_win_count = []
list_total_count = []

for index, row in card_data.iterrows():
    if row['result'] == True:
        list_win_count.append(1)
    else:
        list_win_count.append(0)
    list_total_count.append(1)

clan_data['win_count'] = list_win_count
clan_data['total_count'] = list_total_count

clan_data_result = clan_data.groupby(by = ['main_class', 'sub_class', 'main_index', 'sub_index']).sum().reset_index()
clan_data_result['win_rate'] = clan_data_result['win_count']/clan_data_result['total_count']
clan_data_result.to_csv("clan_data_result.csv",index=False)
